In [127]:
import pandas as pd

## Merge data characteristics and regression results.

In [133]:
df_regression = pd.read_excel("../stats/final_table.xlsx")
bx_age = pd.read_csv("../stats/stats_BX_age.csv")
ml1m_age = pd.read_csv("../stats/stats_ml1m_age.csv")
ml1m_gender = pd.read_csv("../stats/stats_ml1m_gender.csv")

df_regression.fillna(0, inplace=True)

In [134]:
bx_age["Sensitive Feature"] = "Age"
ml1m_age["Sensitive Feature"] = "Age"
ml1m_gender["Sensitive Feature"] = "Gender"

In [135]:
df_concat = pd.concat([bx_age, ml1m_age, ml1m_gender], axis=0, ignore_index=True)

In [136]:
df_final = df_regression.merge(
    df_concat, 
    how="left", 
    on=['Dataset', 'Subset ID', 'Is Filtered', 'Sensitive Feature']
)

In [137]:
df_final.head()

,Model Name,Dataset,Subset ID,Is Filtered,Sensitive Feature,ndcg@5,recall@5,mrr@5,Differential Fairness of sensitive attribute,giniindex@5,...,Standart Deviation of Long Tail Items,Skewness of Long Tail Items,Kurtosis of Long Tail Items,Mean Rating,Standart Deviation of Rating,Skewness of Rating,Kurtosis of Rating,Sensitive Attribute == 0 Percentage,Sensitive Attribute == 1 Percentage,Difference between Sensitive Attribute Percentage
0,FOCF,BX,1,Yes,Age,0.189,0.258,0.182,2.115,0.941,...,0.202,1.576,2.843,1.992,1.529,1.035,-0.689,39.46,60.54,21.08
1,FOCF,BX,2,Yes,Age,0.192,0.264,0.186,3.482,0.948,...,0.175,1.878,4.524,2.005,1.534,1.013,-0.737,39.79,60.21,20.42
2,FOCF,BX,3,Yes,Age,0.209,0.284,0.199,3.293,0.942,...,0.191,1.791,3.828,2.029,1.547,0.980,-0.806,39.16,60.84,21.68
3,FOCF,BX,4,Yes,Age,0.199,0.266,0.195,2.533,0.948,...,0.194,1.673,3.245,1.997,1.531,1.027,-0.708,39.48,60.52,21.04
4,FOCF,BX,5,Yes,Age,0.190,0.249,0.197,2.733,0.955,...,0.146,1.736,4.651,2.013,1.539,1.001,-0.763,39.11,60.89,21.78


In [126]:
df_final.to_csv("df_regression.csv", index=False)

In [140]:
df_final[df_final["Value Unfairness of sensitive attribute"]>5]["Value Unfairness of sensitive attribute"]

207    inf
Name: Value Unfairness of sensitive attribute, dtype: float64